<a href="https://colab.research.google.com/github/junyamahira/ReinforcementLearning_Prac/blob/master/DQN_MountainCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# git clone 
- pipかどちらか実行すればよい
- keras-rlの中に入り、setup.pyを実行する

In [2]:
!git clone https://github.com/keras-rl/keras-rl.git

Cloning into 'keras-rl'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1710 (delta 3), reused 7 (delta 2), pack-reused 1698
Receiving objects: 100% (1710/1710), 1.38 MiB | 19.35 MiB/s, done.
Resolving deltas: 100% (1056/1056), done.


In [11]:
!ls keras-rl

assets		 examples	    mkdocs.yml	rl	   tests
CONTRIBUTING.md  ISSUE_TEMPLATE.md  pytest.ini	setup.cfg  utils
docs		 LICENSE	    README.md	setup.py


In [27]:
!cd keras-rl
!ls

keras-rl  sample_data


# pip

In [28]:
pip install keras-rl

    100% |████████████████████████████████| 40kB 3.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl


# import

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
import gym
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory